In [2]:
import os
import scipy
import numpy as np
import pandas as pd
import sympy as sp

from matplotlib import pyplot as plt

%matplotlib inline

# 4.1

In [3]:
class Moth():
    def __init__(self, X = [85, 196, 341, 578], num_iteration=100) -> None:
        self.X = np.array(X)
        self.sum_ = np.sum(X)
        self.n = np.zeros(9)
        self.p = 1/3*np.ones(3)
        self.num_iteration = num_iteration

    def E_step(self, x, p):
        dom_p1 = p[0]**2+2*p[0]*p[1]+2*p[0]*p[2]
        dom_p2 = p[1]**2+2*p[1]*p[2]
        dom_p3 = dom_p2+p[2]**2

        n_cc = x[0]*p[0]**2/dom_p1
        n_ci = 2*x[0]*p[0]*p[1]/dom_p1
        n_ct = 2*x[0]*p[0]*p[2]/dom_p1
        n_ii = x[1]*p[1]**2/dom_p2
        n_it = 2*x[1]*p[1]*p[2]/dom_p2
        n_tt = x[2]
        n_uii = x[3]*p[1]**2/dom_p3
        n_uit = 2*x[3]*p[1]*p[2]/dom_p3
        n_utt = x[3]*p[2]**2/dom_p3

        n = np.array([n_cc, n_ci, n_ct, n_ii, n_it, n_tt, n_uii, n_uit, n_utt])
        
        return n
    
    def M_step(self, x, n):
        sum_ = np.sum(x)
        p_c = (2*n[0]+n[1]+n[2])/(2*sum_)
        p_i = (2*n[3]+n[4]+n[1]+2*n[6]+n[7])/(2*sum_)
        p_t = (2*n[5]+n[4]+n[2]+2*n[8]+n[7])/(2*sum_)
        p = np.array([p_c, p_i, p_t])

        return p
    
    def l(self, x, p):
        return x[0]*np.log(p[0]**2 + 2*p[0]*p[1] + 2*p[0]*p[2]) + x[1]*np.log(p[1]**2+2*p[1]*p[2]) + 2*x[2]*np.log(p[2]) + 2*x[3]*np.log(p[1]+p[2])
    
    def grad_Q(self, n, p):
        dp_c = (2*n[0]+n[1]+n[2])/p[0] - (n[2]+n[4]+n[7]+2*(n[5]+n[8]))/p[2]
        dp_i = (n[1]+2*(n[3]+n[6])+n[4]+n[7])/p[1] - (n[2]+n[4]+n[7]+2*(n[5]+n[8]))/p[2]
        return np.array([dp_c, dp_i])
    
    def hessian_Q(self, n, p):
        dp_cc = -(2*n[0]+n[1]+n[2])/(p[0]**2) - (n[2]+n[4]+n[7]+2*(n[5]+n[8]))/(p[2]**2)
        dp_ci = -(n[2]+n[4]+n[7]+2*(n[5]+n[8]))/(p[2]**2)
        dp_ii = -(n[1]+2*(n[3]+n[6])+n[4]+n[7])/(p[1]**2) - (n[2]+n[4]+n[7]+2*(n[5]+n[8]))/(p[2]**2)
        return np.array([[dp_cc, dp_ci],[dp_ci, dp_ii]])
    
    def StepHalvingBacktracking(self, x, p0, D):
        t_tmp = 1
        p_dim = p0.shape[0]
        l0 = self.l(x, p0)
        tmp_p = np.zeros(p0.shape)
        tmp_p[:2] = p0[:2] + t_tmp*D
        tmp_p[2] = 1 - np.sum(tmp_p[:2])
        
        while((np.sum(tmp_p >= 0)<p_dim) or (np.sum(tmp_p <= 1)<p_dim)):
            t_tmp = 0.5*t_tmp
            tmp_p = np.zeros(p0.shape)
            tmp_p[:2] = p0[:2] + t_tmp*D
            tmp_p[2] = 1 - np.sum(tmp_p[:2])

        while(self.l(x, tmp_p) < l0):
            t_tmp = 0.5*t_tmp
            tmp_p = np.zeros(p0.shape)
            tmp_p[:2] = p0[:2] + t_tmp*D
            tmp_p[2] = 1 - np.sum(tmp_p[:2])

        return tmp_p
    
    def gradient_M_step(self, x, n, p):
        tmp_p = p.copy()
        hessian_inv = np.linalg.inv(self.hessian_Q(n, tmp_p))
        grad = self.grad_Q(n, tmp_p)
        delta__nt = -hessian_inv@grad
        tmp_p = self.StepHalvingBacktracking(x, tmp_p, delta__nt)

        return tmp_p

    def EM_algorithm(self, X, epsilon=1e-10, inplace=False):
        n, p = self.n, self.p
        for i in range(1, self.num_iteration+1, 1):
            tmp_n = self.E_step(X, p)
            tmp_p = self.M_step(X, tmp_n)
            delta_n = tmp_n - n
            delta_p = tmp_p - p
            p, n = tmp_p, tmp_n
            error = np.sqrt(np.sum(delta_n**2)) + np.sqrt(np.sum(delta_p**2))
            if(error < epsilon):
                break
        if(error >= epsilon):
            print("算法没收敛")
        if(inplace):
            self.p, self.n = p, n

        return p
    
    def BootStrap_p(self, num_bootstrap):
        P = np.zeros((num_bootstrap, self.p.shape[0]))
        for i in range(num_bootstrap):
            n_c = np.random.binomial(self.sum_, self.X[0]/self.sum_, size=None)
            n_i = np.random.binomial(self.sum_-n_c, self.X[1]/(self.sum_-n_c), size=None)
            n_t = np.random.binomial(self.sum_-n_c-n_i, self.X[2]/(self.sum_-n_c-n_i), size=None)
            n_u = self.sum_-n_c-n_t-n_i

            x_generated = np.array([n_c, n_i, n_t, n_u])
            p = self.EM_algorithm(x_generated, inplace=False)
            P[i] = p
            
        return P
    
    def cal_p_statistic(self, num_bootstrap):
        P = self.BootStrap_p(num_bootstrap)
        p_mean = np.mean(P, axis=0)
        p_std = np.std(P, axis=0)
        cov = np.cov(P.T)
        cor = np.corrcoef(P.T)
        return p_mean, p_std, cov, cor
    
    def EM_gradient_algorithm(self, X, epsilon=1e-10, inplace=False):
        n, p = self.n, self.p
        for i in range(1, self.num_iteration+1, 1):
            tmp_n = self.E_step(X, p)
            tmp_p = self.gradient_M_step(X, tmp_n, p)
            delta_n = tmp_n - n
            delta_p = tmp_p - p
            p, n = tmp_p, tmp_n
            error = np.sqrt(np.sum(delta_n**2)) + np.sqrt(np.sum(delta_p**2))
            if(error < epsilon):
                break
        if(error >= epsilon):
            print("算法没收敛")
        if(inplace):
            self.p, self.n = p, n
        return p

In [4]:
X = [85, 196, 341, 578]
moth = Moth(X, num_iteration=100)

In [5]:
MLE_EM = moth.EM_algorithm(X, inplace=True)
print("EM算法得到的p的MLE为: p_c = %f, p_i = %f, p_t = %f"%(MLE_EM[0], MLE_EM[1], MLE_EM[2]))

EM算法得到的p的MLE为: p_c = 0.036067, p_i = 0.195799, p_t = 0.768134


In [6]:
num_iterations = 10000
p_mean, p_std, p_cov, p_cor = moth.cal_p_statistic(num_iterations)

In [7]:
std_p_c, std_p_i, std_p_t = np.sqrt(p_cov[0,0]), np.sqrt(p_cov[1,1]), np.sqrt(p_cov[2,2])
print("Boot strapping采样%d次后得到:"%(num_iterations))
print("p_c, p_i, p_t的标准差分别为%f, %f, %f"%(std_p_c, std_p_i, std_p_t))
print("p的相关系数矩阵为:\n", p_cor)
print("可见, corr(p_c, p_i) = %f, corr(p_c, p_t) = %f, corr(p_i, p_t) = %f"%(p_cor[0,1], p_cor[0,2], p_cor[1,2]))

Boot strapping采样10000次后得到:
p_c, p_i, p_t的标准差分别为0.003865, 0.010986, 0.011390
p的相关系数矩阵为:
 [[ 1.         -0.06959851 -0.27221216]
 [-0.06959851  1.         -0.94095835]
 [-0.27221216 -0.94095835  1.        ]]
可见, corr(p_c, p_i) = -0.069599, corr(p_c, p_t) = -0.272212, corr(p_i, p_t) = -0.940958


In [8]:
MLE_EM_grad = moth.EM_gradient_algorithm(X, inplace=False)
print("EM gradient算法得到的p的MLE为: p_c = %f, p_i = %f, p_t = %f"%(MLE_EM_grad[0], MLE_EM_grad[1], MLE_EM_grad[2]))

EM gradient算法得到的p的MLE为: p_c = 0.036067, p_i = 0.195799, p_t = 0.768134


# 4.2

In [58]:
class HIV():
    def __init__(self, X = [379,299,222,145,109,95,73,59,45,30,24,12,4,2,0,1,1], num_iteration=10000) -> None:
        from scipy.special import gamma
        self.X = np.array(X)
        self.sum_ = np.sum(X)
        self.alpha, self.beta, self.mu, self.lamda = 1/3, 1/3, 0.5, 0.5
        self.theta = (self.alpha, self.beta, self.mu, self.lamda)

        self.I = np.arange(0, self.X.shape[0], 1)
        self.zi = np.zeros(self.I.shape[0])
        self.ti = np.zeros(self.I.shape[0])
        self.pi = np.zeros(self.I.shape[0])

        self.F_gamma = gamma
        self.num_iteration = num_iteration

    def E_step(self, theta):
        alpha, beta, mu, lamda = theta

        I = np.arange(0, 17, 1)
        zi = alpha*((I==0).astype(np.int32))
        ti = beta*np.exp(-mu)*(mu**I)
        pi = (1-alpha-beta)*np.exp(-lamda)*(lamda**I)
        PI = zi+ti+pi

        zi /= PI
        ti /= PI
        pi /= PI

        return I, zi, ti, pi

    def M_step(self, x, p):
        I, zi, ti, pi = p

        sum_ = np.sum(x)
        alpha = np.sum(x[0]*zi)/sum_
        beta = np.sum(x*ti)/sum_
        mu = np.sum(I*x*ti)/np.sum(x*ti)
        lamda = np.sum(I*x*pi)/np.sum(x*pi)
        theta = (alpha, beta, mu, lamda)
        
        return theta

    def EM_algorithm(self, X, epsilon=1e-10, inplace=False):
        theta = (self.alpha, self.beta, self.mu, self.lamda)
        p = self.I, self.zi, self.ti, self.pi

        for i in range(1, self.num_iteration+1, 1):
            tmp_p = self.E_step(theta)
            tmp_theta = self.M_step(X, tmp_p)
            delta_theta = np.array(tmp_theta) - np.array(theta)
            delta_p = np.concatenate([tmp_p[j] - p[j] for j in range(len(tmp_p))])
            error = np.sqrt(np.sum(delta_theta**2)) + np.sqrt(np.sum(delta_p**2))
            p, theta = tmp_p, tmp_theta
            if(error < epsilon):
                break
        if(error >= epsilon):
            print("算法没收敛")
        if(inplace):
            self.I, self.zi, self.ti, self.pi = p
            self.theta = theta

        return theta
    
    def BootStrap_theta(self, num_bootstrap):
        P = np.zeros((num_bootstrap, len(self.theta)))
        for i in range(num_bootstrap):
            x_generated = np.zeros(self.X.shape)
            x_sampled = np.random.choice(np.arange(self.X.shape[0]), np.sum(self.X), p=self.X/np.sum(self.X))
            for j in range(x_generated.shape[0]):
                x_generated[j] = np.where(x_sampled == j)[0].shape[0]
            theta = self.EM_algorithm(x_generated, inplace=False)
            P[i] = np.array(theta)
            
        return P
    
    def cal_p_statistic(self, num_bootstrap):
        P = self.BootStrap_theta(num_bootstrap)
        p_mean = np.mean(P, axis=0)
        p_std = np.std(P, axis=0)
        cov = np.cov(P.T)
        cor = np.corrcoef(P.T)
        return p_mean, p_std, cov, cor

In [59]:
data = np.array([379,299,222,145,109,95,73,59,45,30,24,12,4,2,0,1,1])
hiv = HIV(data)

In [60]:
theta_pred = hiv.EM_algorithm(data, inplace=True)
print("预测的alpha = %f, beta = %f, mu = %f, lambda = %f"%theta_pred)

预测的alpha = 0.122166, beta = 0.562542, mu = 1.467475, lambda = 5.938889


In [61]:
num_iterations = 2000
theta_mean, theta_std, theta_cov, theta_cor = hiv.cal_p_statistic(num_iterations)

In [66]:
std_alpha, std_beta, std_mu, std_lambda = np.sqrt(theta_cov[0,0]), np.sqrt(theta_cov[1,1]), np.sqrt(theta_cov[2,2]), np.sqrt(theta_cov[3,3])
print("Boot strapping采样%d次后得到:"%(num_iterations))
print("alpha, beta, mu, lambda的标准差分别为%f, %f, %f, %f"%(std_alpha, std_beta, std_mu, std_lambda))
print("theta的相关系数矩阵为:\n", theta_cor)
print("可见,\ncorr(alpha, beta) = %f,\tcorr(alpha, mu) = %f,\tcorr(alpha, lambda) = %f,\ncorr(beta, mu) = %f,\tcorr(beta, lambda) = %f,\tcorr(mu, lambda) = %f"%(theta_cor[0,1], theta_cor[0,2], theta_cor[0,3], theta_cor[1,2], theta_cor[1,3], theta_cor[2,3]))

Boot strapping采样2000次后得到:
alpha, beta, mu, lambda的标准差分别为0.044415, 0.089679, 1.325910, 1.452198
theta的相关系数矩阵为:
 [[ 1.         -0.59179758  0.39238327 -0.40846613]
 [-0.59179758  1.         -0.94386978  0.9611617 ]
 [ 0.39238327 -0.94386978  1.         -0.9805038 ]
 [-0.40846613  0.9611617  -0.9805038   1.        ]]
可见,
corr(alpha, beta) = -0.591798,	corr(alpha, mu) = 0.392383,	corr(alpha, lambda) = -0.408466,
corr(beta, mu) = -0.943870,	corr(beta, lambda) = 0.961162,	corr(mu, lambda) = -0.980504
